In [1]:
import torch
import torchvision.transforms as transforms
from LookGenerator.datasets.tps_dataset import ShirtsDataset
from torch.utils.data import DataLoader
from LookGenerator.networks.bpgm.model.models import BPGM
from LookGenerator.networks.bpgm.train_bpgm import train_bpgm
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from LookGenerator.networks_training.utils import check_path_and_creat
from LookGenerator.networks.utils import save_model

In [2]:
transform = transforms.Compose([
    transforms.Resize((256,192)),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])
mask_transform = transforms.Compose([
    transforms.Resize((256,192)),
])


In [3]:
batch_size = 32
num_workers = 0
pin_memory = True

In [4]:
dataset = ShirtsDataset(
    root=r"C:\Users\DenisovDmitrii\Desktop\zalando-hd-resize\train",
    transform=transform,
    transform_mask=mask_transform
)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, pin_memory=pin_memory, num_workers=num_workers)

In [5]:
save_directory = r'C:\Users\DenisovDmitrii\OneDrive - ITMO UNIVERSITY\peopleDetector\tps\weights\test'
check_path_and_creat(save_directory)

True

In [6]:
for data in dataloader:
    break

C:\Users\DenisovDmitrii\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [7]:
model = BPGM()

initialization method [normal]
initialization method [normal]


In [10]:
device = "cpu" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [11]:
loss = train_bpgm(dataloader=dataloader,
                  model=model, device=device, epochs=1)

  0%|          | 0/364 [00:00<?, ?it/s]

<class 'torch.Tensor'> cpu


  0%|          | 1/364 [00:12<1:13:47, 12.20s/it]

<class 'torch.Tensor'> cpu


  1%|          | 2/364 [00:24<1:14:31, 12.35s/it]

<class 'torch.Tensor'> cpu


  1%|          | 2/364 [00:32<1:36:37, 16.01s/it]


KeyboardInterrupt: 

In [ ]:
save_model(model.to('cpu'), path=f"{save_directory}\\epoch_00.pt")

# тест

In [8]:
test_dataset = ShirtsDataset(
    root=r"C:\Users\DenisovDmitrii\Desktop\zalando-hd-resize\train",
    transform=transform,
    transform_mask=mask_transform
)

In [9]:
topil = transforms.ToPILImage()

In [23]:
testdataloader = DataLoader(test_dataset, batch_size=1, pin_memory=True)

In [24]:
for data in testdataloader:
    mask = data['segmentation']
    shirt = data['shirt']
    theta = model(mask, shirt)

    warped = F.grid_sample(shirt, theta, padding_mode='border', align_corners=True)

    warped = topil(warped[0]/2+0.5)
    warped.show()
    break